# AugmentedGPT Playground

<a target="_blank" href="https://colab.research.google.com/github/wenyuzhao/augmented-gpt/blob/main/examples/example.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [1]:
%pip install augmented-gpt --quiet

Note: you may need to restart the kernel to use updated packages.


## Imports and set API key

In [1]:
from augmented_gpt import *
import getpass
AugmentedGPT.set_api_key(getpass.getpass('Enter your OpenAI API key: '))

## Create a GPT instance that can use tools, and remembers the conversation in a long-term memory storage.

In [2]:
@function
def send_email(
    topic: str = param("The email topic"),
    to: str = param("The recipient email address"),
    content: str = param("The content of the email"),
):
    """Send an email to a recipient"""
    gpt.logger.debug(f"➡️ send_email to {to}")
    print(f"---")
    print(f"TOPIC: {topic}")
    print(f"TO: {to}")
    print(f"CONTENT:\n\n{content}")
    print(f"---")
    return "done."

gpt = AugmentedGPT(functions=[send_email], plugins=plugins.all_plugins(), debug=True)

gpt.get_plugin("Memory").clear_memory() # type: ignore

completion = gpt.chat_completion([
    Message(role=Role.USER, content="Steve was wondering what is the distance between moon and earth \
        divided by two. Figure this out and send him an email."),
])

async for response in completion:
    print(response)

[INFO:AugmentedGPT] Register-Function: send_email
[INFO:AugmentedGPT] Register-Function: Timestamp-get_current_timestamp
[INFO:AugmentedGPT] Register-Function: Memory-remember
[INFO:AugmentedGPT] Register-Function: Memory-search_from_memory
[INFO:AugmentedGPT] Register-Function: Calculator-evaluate


Message(role='assistant', content=None, name='Calculator-evaluate', function_call=FunctionCall(name='Calculator-evaluate', arguments={'expression': '384400 / 2'}))


[DEBUG:AugmentedGPT] ➡️ Calculator-evaluate: expression=384400 / 2


Message(role='function', content='192200.0', name='Calculator-evaluate', function_call=None)
Message(role='assistant', content=None, name='send_email', function_call=FunctionCall(name='send_email', arguments={'topic': 'Distance between Moon and Earth divided by two', 'to': 'steve@example.com', 'content': 'The distance between the Earth and the Moon divided by two is 192,200 kilometers.'}))


[DEBUG:AugmentedGPT] ➡️ send_email: topic=Distance between Moon and Earth divided by two, to=steve@example.com, content=The distance between the Earth and the Moon divided by two is 192,200 kilometers.
[DEBUG:AugmentedGPT] ➡️ send_email to steve@example.com


---
TOPIC: Distance between Moon and Earth divided by two
TO: steve@example.com
CONTENT:

The distance between the Earth and the Moon divided by two is 192,200 kilometers.
---
Message(role='function', content='done.', name='send_email', function_call=None)
Message(role='assistant', content="I've sent an email to Steve.", name=None, function_call=None)


## Create another GPT instance that inherits and utilizes the memory.

In [ ]:
gpt = AugmentedGPT(functions=[send_email], plugins=plugins.all_plugins(), debug=True)

# Make it retrieve the answer from the memorized conversation history
completion = gpt.chat_completion([
    Message(role=Role.USER, content="Ok Steve asked for the Moon and Earth distance question again \
        after 10 days! Email him the answer if you can still remember, and tell him \
        he already asked for this before!"),
])

async for message in completion:
    print(message)

[INFO:AugmentedGPT] Register-Function: send_email
[INFO:AugmentedGPT] Register-Function: Timestamp-get_current_timestamp
[INFO:AugmentedGPT] Register-Function: Memory-remember
[INFO:AugmentedGPT] Register-Function: Memory-search_from_memory
[INFO:AugmentedGPT] Register-Function: Calculator-evaluate


Message(role='assistant', content=None, name=None, function_call={'name': 'Memory-search_from_memory', 'arguments': '{\n  "query": "moon and earth distance"\n}'})


[DEBUG:AugmentedGPT] ➡️ search_from_memory: moon and earth distance


Message(role='function', content='TIMESTAMP: 2023-06-28T08:52:19.787939\nCONTENT:\n{"role": "assistant", "content": "I have figured out the distance between the Moon and Earth divided by two. It is approximately 192,200 kilometers. I have sent an email to Steve with this information."}\n\n---\n\nTIMESTAMP: 2023-06-28T08:52:13.823684\nCONTENT:\n{"role": "user", "content": "Steve was wondering what is the distance between moon and earth         divided by two. Figure this out and send him an email."}\n\n---\n\nTIMESTAMP: 2023-06-28T08:52:30.869264\nCONTENT:\n{"role": "user", "content": "Ok Steve asked for the Moon and Earth distance question again         after 10 days! Email him the answer if you can still remember, and tell him         he already asked for this before!"}\n\n---\n\n', name='Memory-search_from_memory', function_call=None)
Message(role='assistant', content=None, name=None, function_call={'name': 'send_email', 'arguments': '{\n  "topic": "Moon and Earth Distance",\n  "to":

[DEBUG:AugmentedGPT] ➡️ send_email to steve@example.com


---
TOPIC: Moon and Earth Distance
TO: steve@example.com
CONTENT:

Hey Steve,

I remember you asked me about the distance between the Moon and Earth divided by two before. The approximate distance is 192,200 kilometers. If you have any other questions, feel free to ask!

Best regards,
Assistant
---
Message(role='function', content='done.', name='send_email', function_call=None)
Message(role='assistant', content='I have emailed Steve with the answer to his question about the distance between the Moon and Earth divided by two. I also mentioned in the email that he had asked this question before.', name=None, function_call=None)
